In [1]:
# 一整頁文章一起跑
import re
import requests
import time
import math
import numpy as np
import operator
from bs4 import BeautifulSoup as bs
from collections import Counter
import os

if not os.path.exists('./Workspace/data/dictionary'):
    os.makedirs('./Workspace/data/dictionary')
if not os.path.isfile('./Workspace/data/dictionary/autodict_v1.txt'):
    with open('./Workspace/data/dictionary/autodict_v1.txt', 'w'):
        pass
    
#移除標點符號
def removePunctuation(source):
    xx = u"([^\u4e00-\u9fff]+)"
    s = re.sub(xx,' ',source)
    return s

#移除標籤，這邊沒用到
def removeTag(soup,tag):
    [x.extract() for x in soup.select(tag)]
        
#計算entropy，偷來改的，懶得改輸入型態，他是設計成list輸入，可一次輸出多Ngram的entropy，我只用單值
def entropy(n_gram_lengths, text):
    entropies = {}
    #可逐一將list的n代入，不過我都一次只丟一個n進去
    for n in n_gram_lengths:
        #開始做n-gram
        length_n_substrings = [text[i:i+n] for i in range(len(text)-n+1)]
        #計算字頻
        length_n_substring_counts = Counter(length_n_substrings)
        #斷出來的總數
        normalizer = float(len(length_n_substrings))
        #標準化
        distribution_values = np.array(length_n_substring_counts.values()) / normalizer
        #計算entropy
        entropy = -np.sum(p*np.log2(p) for p in distribution_values)
        #把結果丟進list
        entropies[n] = entropy
    #輸出    
    return entropies


#抓蘋果即時新聞
def getAll(section, page):
    domain = 'http://www.appledaily.com.tw'
    res = requests.get('http://www.appledaily.com.tw/realtimenews/section/%s/%d' % (section, page))
    soup = bs(res.text, 'html.parser')
    urls = [domain+t['href'] for t in soup.select('.rtddt a') if 'appledaily' not in t['href']]
    #用url來爬內容
    return getContent(urls)

#爬內容
def getContent(urls):
    news = ''
    for u in urls:
        res = requests.get(u)
        soup = bs(res.text,'html.parser')
        apple = soup.select('#summary')[0].text
        #文章合併
        news += removePunctuation(apple)
    return news


#頁數範圍
for p in xrange(1,57):
    #獲取一整頁的文章
    news = getAll('new',p)
    #斷句
    sp = news.split()
    #跑2-gram至4-gram
    for n in xrange(2,5):
        #存字頻的字典
        substring_counts_dict = {}
        #逐句做n-gram
        for s in sp:
            sentence_substrings = [s[i:i+n] for i in range(len(s)-n+1)]
            #計算字頻
            for word in sentence_substrings:
                if word not in substring_counts_dict:
                    substring_counts_dict[word] = 1
                else:
                    substring_counts_dict[word] += 1
        #字頻排序
        word_freq = sorted(substring_counts_dict.iteritems(),key=operator.itemgetter(1),reverse=True)
        #計算完整entropy
        org = entropy([n], news).values()[0]
        #空字典，拿來存排除斷詞後的entropy
        word_entropy = {}
        #留下字頻大於5的斷詞
        word_freq = [w for w in word_freq if w[1]>=5]
        #分別將斷詞結果刪除，計算entropy
        for w in word_freq:
            word_entropy[w[0]] = entropy([n], news.replace(w[0],'')).values()[0]
            #單純用字頻做限制的結果
            #print w[0], w[1]
        #用entropy排序
        weighted_world = sorted(word_entropy.iteritems(),key=operator.itemgetter(1),reverse=False)
        #逐一檢查篩選後之斷詞結果，留下entropy變化比例>=0.0001的結果
        for keyword in weighted_world:
            if (org-keyword[1])/org >= 0.0001:
                with open('./Workspace/data/dictionary/autodict_v1.txt', 'a') as f:
                    #輸出資訊變化比重
                    #f.write('%s %s %s' % (keyword[0].encode('utf-8'), str((org-keyword[1])/org), '\n'))
                    f.write('%s%s' % (keyword[0].encode('utf-8'), '\n'))